# Computing the PCA of a Foreground Object

### Setup

Let's start by loading some pre-requisites and checking the DINOv3 repository location:
- `local` if `DINOV3_LOCATION` environment variable was set to work with a local version of DINOv3 repository;
- `github` if the code should be loaded via torch hub.

In [ ]:
import pickle
import os
import urllib

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision.transforms.functional as TF
from sklearn.decomposition import PCA
from scipy import signal

import random
import tqdm # Progress bar
from sklearn.metrics import pairwise_distances # Isso aqui é para calcular a distancia média entre os patches

DINOV3_GITHUB_LOCATION = "/home/lades/computer_vision/wesley/dino-soja/dinov3"

if os.getenv("DINOV3_LOCATION") is not None:
    DINOV3_LOCATION = os.getenv("DINOV3_LOCATION")
else:
    DINOV3_LOCATION = DINOV3_GITHUB_LOCATION

print(f"DINOv3 location set to {DINOV3_LOCATION}")

# examples of available DINOv3 models:
MODEL_DINOV3_VITS = "dinov3_vits16"
MODEL_DINOV3_VITSP = "dinov3_vits16plus"
MODEL_DINOV3_VITB = "dinov3_vitb16"
MODEL_DINOV3_VITL = "dinov3_vitl16"
MODEL_DINOV3_VITHP = "dinov3_vith16plus"
MODEL_DINOV3_VIT7B = "dinov3_vit7b16"

MODEL_NAME = MODEL_DINOV3_VITL

print(f"Loading DINOv3 model {MODEL_NAME}...")

model = torch.hub.load(
    repo_or_dir=DINOV3_LOCATION,
    model=MODEL_NAME,
    source="local",
    weights="https://huggingface.co/MVRL/dinov3_vitl16_sat/resolve/main/dinov3_vitl16_pretrain_sat493m-eadcf0ff.pth"
)
model.cuda()

PATCH_SIZE = 16
IMAGE_SIZE = 256

SATELLITE_MEAN = (0.430, 0.411, 0.296)
SATELLITE_STD = (0.213, 0.156, 0.143)

MODEL_TO_NUM_LAYERS = {
    MODEL_DINOV3_VITS: 12,
    MODEL_DINOV3_VITSP: 12,
    MODEL_DINOV3_VITB: 12,
    MODEL_DINOV3_VITL: 24,
    MODEL_DINOV3_VITHP: 32,
    MODEL_DINOV3_VIT7B: 40,
}

n_layers = MODEL_TO_NUM_LAYERS[MODEL_NAME]
    
def load_image(path: str) -> Image:
    return Image.open(path).convert("RGB")

# image resize transform to dimensions divisible by patch size
def resize_transform(
    mask_image: Image,
    image_size: int = IMAGE_SIZE,
    patch_size: int = PATCH_SIZE,
) -> torch.Tensor:
    w, h = mask_image.size
    h_patches = int(image_size / patch_size)
    w_patches = int((w * image_size) / (h * patch_size))
    return TF.to_tensor(TF.resize(mask_image, (h_patches * patch_size, w_patches * patch_size)))

DINOv3 location set to /home/lades/computer_vision/wesley/dino-soja/dinov3
Loading DINOv3 model dinov3_vitl16...


In [5]:
# Obter imagem aleatória do dataset
dataset_path = "/home/lades/computer_vision/wesley/dataset/daninhas_multiclasse/DATASET_CARURU/rgb/"
image_uri = f"{dataset_path}{random.choice(os.listdir(dataset_path))}"

print(f"Using image {image_uri}")

# Obtém o respectivo label
label_uri = image_uri.replace("rgb", "labels").replace(".jpg", ".png")
label = np.array(Image.open(label_uri))

print(f"Using label {label_uri}")
np.unique(label)

Using image /home/lades/computer_vision/wesley/dataset/daninhas_multiclasse/DATASET_CARURU/rgb/2025-09-17_TH134_06-05-2025_c_mask_14_8807_24073.jpg
Using label /home/lades/computer_vision/wesley/dataset/daninhas_multiclasse/DATASET_CARURU/labels/2025-09-17_TH134_06-05-2025_c_mask_14_8807_24073.png


array([  0,   1, 255], dtype=uint8)

In [7]:
def calcular_pca(label, image_uri): 
    # Cria a máscara de foreground (tudo que estiver no fundo preto que seja diferente de branco)
    label_mask = (label == 1).astype(np.float32)
    label_mask = signal.convolve2d(label_mask, np.ones((PATCH_SIZE, PATCH_SIZE)), mode='valid')[::PATCH_SIZE, ::PATCH_SIZE]
    fg_score_mf = torch.from_numpy(label_mask > 0.5)

    image = load_image(image_uri)
    image_resized = resize_transform(image)
    image_resized_norm = TF.normalize(image_resized, mean=SATELLITE_MEAN, std=SATELLITE_STD)

    h_patches, w_patches = image_resized_norm.shape[1] // PATCH_SIZE, image_resized_norm.shape[2] // PATCH_SIZE
    print(f"Image size: {image_resized_norm.shape}, patches: {h_patches}x{w_patches}")

    with torch.inference_mode():
        with torch.autocast(device_type='cuda', dtype=torch.float32):
            feats = model.get_intermediate_layers(image_resized_norm.unsqueeze(0).cuda(), n=range(n_layers), reshape=True, norm=True)
            x = feats[-1].squeeze().detach().cpu()
            dim = x.shape[0]
            x = x.view(dim, -1).permute(1, 0)
            
    fg_patches = x

    pca = PCA(n_components=3, whiten=True)
    pca.fit(fg_patches)

    # apply the PCA, and then reshape
    projected_image = torch.from_numpy(pca.transform(x.numpy())).view(h_patches, w_patches, 3)

    # multiply by 2.0 and pass through a sigmoid to get vibrant colors 
    projected_image = torch.nn.functional.sigmoid(projected_image.mul(2.0)).permute(2, 0, 1)

    # mask the background using the fg_score_mf
    projected_foreground_image = projected_image * (fg_score_mf.unsqueeze(0) > 0.5)
    
    projected_background_image = projected_image * (fg_score_mf.unsqueeze(0) <= 0.5)

    return projected_image, projected_foreground_image, projected_background_image

def plotar_imagens(image_uri, projected_image, projected_foreground_image, label_mask, label_uri):
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 5, 1)
    plt.imshow(np.array(Image.open(image_uri)))
    plt.title("Imagem original")
    plt.subplot(1, 5, 2)
    plt.imshow(projected_image.permute(1, 2, 0))
    plt.title("PCA da imagem projetada")
    plt.subplot(1, 5, 3)
    plt.imshow(projected_foreground_image.permute(1, 2, 0))
    plt.title("PCA da imagem projetada + mask")
    plt.subplot(1, 5, 4)
    plt.imshow(np.array(Image.fromarray(label_mask)))
    plt.title("PCA da mask label (GT)")
    plt.subplot(1, 5, 5)
    plt.imshow(Image.open(label_uri))
    plt.title("Label (GT)")
    plt.show()

    # Exibir label_mask com os valores de cada bloquinho
    plt.figure(figsize=(10, 10))
    plt.imshow(label_mask)
    for i in range(label_mask.shape[0]):
        for j in range(label_mask.shape[1]):
            plt.text(j, i, f"{label_mask[i, j]:.1f}", ha='center', va='center', color='white')
    plt.show()

    # Exibir projected_image com os valores de cada bloquinho
    plt.figure(figsize=(10, 10))
    plt.imshow(projected_image.permute(1, 2, 0))
    for i in range(projected_image.shape[1]):
        for j in range(projected_image.shape[2]):
            plt.text(j, i, f"{projected_image[0, i, j]:.1f}", ha='center', va='center', color='white')
    plt.show()

### Experimento
Neste experimento, irei considerar o mask label da respectiva imagem, para separar os patches do PCA da imagem que estiverem dentro da mask label da imagem. Isso sera o foreground e o restante, background.

In [2]:
def extrair_tensores(label, image_uri):
    image = load_image(image_uri)
    image_resized = resize_transform(image)
    image_resized_norm = TF.normalize(image_resized, mean=SATELLITE_MEAN, std=SATELLITE_STD)

    label_mask = (label == 1).astype(np.float32)
    label_mask = signal.convolve2d(label_mask, np.ones((PATCH_SIZE, PATCH_SIZE)), mode='valid')[::PATCH_SIZE, ::PATCH_SIZE]
    fg_score_mf = torch.from_numpy(label_mask > 0.5)

    h_patches, w_patches = image_resized_norm.shape[1] // PATCH_SIZE, image_resized_norm.shape[2] // PATCH_SIZE
    #print(f"Image size: {image_resized_norm.shape}, patches: {h_patches}x{w_patches}")

    with torch.inference_mode():
        with torch.autocast(device_type='cuda', dtype=torch.float32):
            feats = model.get_intermediate_layers(image_resized_norm.unsqueeze(0).cuda(), n=range(n_layers), reshape=True, norm=True)
            x = feats[-1].squeeze().detach().cpu()
            dim = x.shape[0]
            x = x.view(dim, -1).permute(1, 0)
            
    x.shape, fg_score_mf.shape

    # Dividir x em foreground e background, considerando a label_mask
    foreground_x = x[fg_score_mf.view(-1) > 0.5]
    background_x = x[fg_score_mf.view(-1) <= 0.5]

    #print(f'Foreground: {foreground_x.shape}, Background: {background_x.shape}')

    return x, foreground_x, background_x


In [3]:
def agrupar_tensores(dataset_path): # dataset_path deve ser o path do dataset (ex: .../DATASET_CARURU/)
    dataset_path_rgb = f"{dataset_path}/rgb/"
    dataset_path_label = f"{dataset_path}/labels/" # Extensão png
    
    lista_imagens = [f"{dataset_path_rgb}{file}" for file in os.listdir(dataset_path_rgb) if file.endswith('.jpg')]
    lista_labels = [f"{dataset_path_label}{file}" for file in os.listdir(dataset_path_label) if file.endswith('.png')]
    
    lista_imagens.sort()
    lista_labels.sort()

    print(f"Foram encontrados {len(lista_imagens)} imagens e {len(lista_labels)} labels.")

    x_foreground = []
    x_background = []

    # Iterar sobre as imagens utilizando tqdm para mostrar o progresso
    for image_uri, label_uri in tqdm.tqdm(zip(lista_imagens, lista_labels), total=len(lista_imagens)):
        label = np.array(Image.open(label_uri))
        x, foreground, background = extrair_tensores(label, image_uri)
        x_foreground.append(foreground)
        x_background.append(background)

    all_foreground = torch.cat(x_foreground, dim=0)
    all_background = torch.cat(x_background, dim=0)
    
    print(f'Total Foreground: {all_foreground.shape}, Total Background: {all_background.shape}')
    
    return all_foreground, all_background

In [4]:
dataset_path = "/home/lades/computer_vision/wesley/dataset/daninhas_multiclasse/"

# Obter os nomes das classes, pelo sufixo DATASET_
dir_classes = [d for d in os.listdir(dataset_path) if d.startswith("DATASET_")]

classes = []
for dir_class in dir_classes:
    class_name = dir_class.split("DATASET_")[-1]
    # nickname sera as 3 primeiras letras (se a classe tiver somente uma palavra); ou as iniciais das palavras (se tiver mais de uma palavra)
    if "_" in class_name:
        nickname = "".join([word[0] for word in class_name.split("_")]).upper()
    else:
        nickname = class_name[:3].upper()
    classes.append((class_name, dir_class, nickname))

classes.sort()

classes

[('CARURU', 'DATASET_CARURU', 'CAR'),
 ('GRAMINEA_PORTE_ALTO', 'DATASET_GRAMINEA_PORTE_ALTO', 'GPA'),
 ('GRAMINEA_PORTE_BAIXO', 'DATASET_GRAMINEA_PORTE_BAIXO', 'GPB'),
 ('MAMONA', 'DATASET_MAMONA', 'MAM'),
 ('OUTRAS_FOLHAS_LARGAS', 'DATASET_OUTRAS_FOLHAS_LARGAS', 'OFL'),
 ('TREPADEIRA', 'DATASET_TREPADEIRA', 'TRE')]

In [ ]:
def calcular_distancias_e_desvio_padrao(all_x_foreground, all_x_background, 
                                 max_samples_fg=5000, max_samples_bg=10000):
    """
    Calcula distâncias usando amostragem para economizar memória
    """
    print(f"Original - FG: {all_x_foreground.shape[0]:,}, BG: {all_x_background.shape[0]:,}")
    
    if all_x_foreground.shape[0] > max_samples_fg:
        # Amostrar foreground
        indices_fg = torch.randperm(all_x_foreground.shape[0])[:max_samples_fg]
        fg_sample = all_x_foreground[indices_fg]
        #print(f"Amostrando foreground: {max_samples_fg:,} patches")
    else:
        fg_sample = all_x_foreground
        
    if all_x_background.shape[0] > max_samples_bg:
        # Amostrar background
        indices_bg = torch.randperm(all_x_background.shape[0])[:max_samples_bg]
        bg_sample = all_x_background[indices_bg]
        #print(f"Amostrando background: {max_samples_bg:,} patches")
    else:
        bg_sample = all_x_background
    
    print(f"Calculando distâncias: {fg_sample.shape[0]:,} x {fg_sample.shape[0]:,} (intra)")
    print(f"Calculando distâncias: {fg_sample.shape[0]:,} x {bg_sample.shape[0]:,} (inter)")
    
    # Calcular distâncias nas amostras
    mean_dist_intra = pairwise_distances(fg_sample).mean()
    mean_dist_inter = pairwise_distances(fg_sample, bg_sample).mean()
    std_intra = pairwise_distances(fg_sample).std()
    std_inter = pairwise_distances(fg_sample, bg_sample).std()

    return mean_dist_intra, mean_dist_inter, std_intra, std_inter

: 

In [5]:
def pairwise_distances_torch(x, y=None):
    """
    Uma função wrapper que usa as funções de distância nativas do PyTorch.
    Ela roda na GPU se os tensores de entrada estiverem na GPU.
    """
    if y is None:
        # torch.pdist calcula a distância par-a-par dentro de um único tensor
        return torch.pdist(x)
    else:
        # torch.cdist calcula a distância entre cada par de vetores de dois tensores
        return torch.cdist(x, y)

def calcular_distancias_otimizado(all_x_foreground, all_x_background, batch_size=256):
    """
    Calcula a média e o desvio padrão das distâncias usando processamento em lotes
    para economizar memória, utilizando 100% dos dados.
    """
    device = all_x_foreground.device # Usa o mesmo dispositivo (CPU ou GPU) dos tensores
    print(f"Iniciando cálculo otimizado com batch size: {batch_size}")
    print(f"Usando dispositivo: {device}")
    
    # --- Cálculo INTRA-CLASSE (foreground vs foreground) ---
    print(f"Calculando distâncias intra-classe para {all_x_foreground.shape[0]:,} vetores...")
    count_intra = 0
    mean_intra = torch.tensor(0.0, device=device)
    M2_intra = torch.tensor(0.0, device=device)

    # Itera sobre os dados em lotes com tqdm
    for i in tqdm.tqdm(range(0, all_x_foreground.shape[0], batch_size), desc="Intra-classe"):
        batch_fg = all_x_foreground[i:i + batch_size]
        
        # Calcula distâncias apenas deste lote contra todos os outros
        dist_batch = pairwise_distances_torch(batch_fg, all_x_foreground)
        
        # Algoritmo de Welford para média e variância online
        for d in dist_batch.flatten(): # Itera sobre cada distância calculada
            count_intra += 1
            delta = d - mean_intra
            mean_intra += delta / count_intra
            delta2 = d - mean_intra
            M2_intra += delta * delta2

    std_intra = torch.sqrt(M2_intra / count_intra) if count_intra > 0 else torch.tensor(0.0)
    
    # --- Cálculo INTER-CLASSE (foreground vs background) ---
    print(f"Calculando distâncias inter-classe para {all_x_foreground.shape[0]:,} vs {all_x_background.shape[0]:,} vetores...")
    count_inter = 0
    mean_inter = torch.tensor(0.0, device=device)
    M2_inter = torch.tensor(0.0, device=device)

    for i in tqdm.tqdm(range(0, all_x_foreground.shape[0], batch_size), desc="Inter-classe"):
        batch_fg = all_x_foreground[i:i + batch_size]
        
        # Calcula distâncias apenas deste lote contra todo o background
        dist_batch = pairwise_distances(batch_fg, all_x_background)
        
        # Algoritmo de Welford
        for d in dist_batch.flatten():
            count_inter += 1
            delta = d - mean_inter
            mean_inter += delta / count_inter
            delta2 = d - mean_inter
            M2_inter += delta * delta2
            
    std_inter = torch.sqrt(M2_inter / count_inter) if count_inter > 0 else torch.tensor(0.0)

    return mean_intra.item(), mean_inter.item(), std_intra.item(), std_inter.item()

In [7]:
def combine_stats(mean_A, M2_A, count_A, mean_B, M2_B, count_B):
    """
    Combina estatísticas de dois conjuntos. A entrada precisa ser dois tensores com média e contagem.
    """
    
    # Iteração inicial
    if count_A == 0:
        return mean_B, M2_B, count_B
    
    if count_B == 0:
        return mean_A, M2_A, count_A
    
    count_total = count_A + count_B
    
    delta = mean_B - mean_A # Diferença
    
    # Média ponderada. N precisa verificar se é maior do que zero, porque já foi verificado antes
    mean_total = (mean_A * count_A + mean_B * count_B) / count_total
    
    # Atualiza M2 usando a fórmula de combinação
    M2_total = M2_A + M2_B + delta * delta * (count_A * count_B) / count_total
    
    return mean_total, M2_total, count_total

def calcular_distancias_otimizado2(all_x_foreground, all_x_background, batch_size=256):
    """
    Esta versão irá calcular as distancias em lotes, utilizando a VRAM (24GB). Como não conseguiremos carregar tudo na VRAM, faremos em lotes.
    A ideia é utilizar a combine_stats para ir acumulando as médias e desvios padrão.
    Essa versão calcula as distancias na VRAM.
    """
    device = all_x_foreground.device # Usa o mesmo dispositivo (CPU ou GPU) dos tensores
    print(f"Iniciando cálculo otimizado com batch size: {batch_size}")
    print(f"Usando dispositivo: {device}")
    
    # --- Cálculo INTRA-CLASSE (foreground vs foreground) ---
    print(f"Calculando distâncias intra-classe para {all_x_foreground.shape[0]:,} vetores...")
    mean_intra = torch.tensor(0.0, device=device, dtype=all_x_foreground.dtype)
    M2_intra = torch.tensor(0.0, device=device, dtype=all_x_foreground.dtype)
    count_intra = torch.tensor(0, device=device, dtype=torch.int64)
    
    # Itera sobre os dados em lotes com tqdm
    for i in tqdm.tqdm(range(0, all_x_foreground.shape[0], batch_size), desc="Intra-classe"):
        batch_fg = all_x_foreground[i:i + batch_size]
                        
        # Calcula distâncias apenas deste lote contra todos os outros
        dist_batch = pairwise_distances_torch(batch_fg, all_x_foreground)
        
        batch_count = torch.tensor(dist_batch.numel(), device=device, dtype=torch.int64)
        
        if batch_count == 0:
            continue

        # Calcula estatísticas do lote atual
        batch_mean = torch.mean(dist_batch)
        
        torch.sub(dist_batch, batch_mean, out=dist_batch)
        torch.pow(dist_batch, 2, out=dist_batch)
        batch_M2 = torch.sum(dist_batch)
        
        # Combina as estatísticas do lote com as acumuladas
        mean_intra, M2_intra, count_intra = combine_stats(mean_intra, M2_intra, count_intra, batch_mean, batch_M2, batch_count)
        
        del dist_batch, batch_mean, batch_M2, batch_count
        torch.cuda.empty_cache()

    std_intra = torch.sqrt(M2_intra / count_intra) if count_intra > 0 else torch.tensor(0.0, device=device, dtype=all_x_foreground.dtype)

    # --- Cálculo INTER-CLASSE (foreground vs background) ---
    print(f"Calculando distâncias inter-classe para {all_x_foreground.shape[0]:,} vs {all_x_background.shape[0]:,} vetores...")
    mean_inter = torch.tensor(0.0, device=device, dtype=all_x_foreground.dtype)
    M2_inter = torch.tensor(0.0, device=device, dtype=all_x_foreground.dtype)
    count_inter = torch.tensor(0, device=device, dtype=torch.int64)
    
    for i in tqdm.tqdm(range(0, all_x_foreground.shape[0], batch_size), desc="Inter-classe"):
        batch_fg = all_x_foreground[i:i + batch_size]
                        
        # Calcula distâncias apenas deste lote contra todo o background
        dist_batch = pairwise_distances_torch(batch_fg, all_x_background)
        
        batch_count = torch.tensor(dist_batch.numel(), device=device, dtype=torch.int64)
        
        if batch_count == 0:
            continue

        # Calcula estatísticas do lote atual
        batch_mean = torch.mean(dist_batch)
        
        torch.sub(dist_batch, batch_mean, out=dist_batch)
        torch.pow(dist_batch, 2, out=dist_batch)
        batch_M2 = torch.sum(dist_batch)
        #batch_M2 = torch.sum((dist_batch - batch_mean) ** 2)
                
        # Combina as estatísticas do lote com as acumuladas
        mean_inter, M2_inter, count_inter = combine_stats(mean_inter, M2_inter, count_inter, batch_mean, batch_M2, batch_count)
        
        del dist_batch, batch_mean, batch_M2, batch_count
        torch.cuda.empty_cache()
        
    std_inter = torch.sqrt(M2_inter / count_inter) if count_inter > 0 else torch.tensor(0.0, device=device, dtype=all_x_foreground.dtype)
    
    return mean_intra.item(), mean_inter.item(), std_intra.item(), std_inter.item()

In [ ]:
# Iterar sobre as classes e criar um array unico. Inicialmente conterá todas as all_foreground, all_background, mean_dist_classe_classe e mean_dist_classe_not_classe por classe

results = []

for i, (class_name, dir_class, nickname) in enumerate(classes):
    print(f"Processando classe {class_name} – {nickname}...")
    all_x_foreground, all_x_background = agrupar_tensores(f"{dataset_path}{dir_class}")
    #mean_dist_classe_classe = pairwise_distances(all_x_foreground).mean()
    #mean_dist_classe_not_classe = pairwise_distances(all_x_foreground, all_x_background).mean()
    #mean_dist_classe_classe, mean_dist_classe_not_classe, std_classe_classe, std_classe_not_classe = calcular_distancias_e_desvio_padrao(all_x_foreground, all_x_background, max_samples_fg=int(0.5*all_x_foreground.shape[0]), max_samples_bg=int(0.5*all_x_background.shape[0]))
    mean_dist_classe_classe, mean_dist_classe_not_classe, std_classe_classe, std_classe_not_classe = calcular_distancias_otimizado(all_x_foreground, all_x_background, batch_size=256)

    results.append({
        "class": nickname,
        "all_foreground": all_x_foreground,
        "all_background": all_x_background,
        "mean_dist_classe_classe": mean_dist_classe_classe,
        "mean_dist_classe_not_classe": mean_dist_classe_not_classe,
        "std_classe_classe": std_classe_classe,
        "std_classe_not_classe": std_classe_not_classe
    })
    print(f"Distância média {nickname}_{nickname}: {mean_dist_classe_classe}, {nickname}_not_{nickname}: {mean_dist_classe_not_classe}")
    print(f"Desvio padrão {nickname}_{nickname}: {std_classe_classe}, {nickname}_not_{nickname}: {std_classe_not_classe}")
    

Processando classe CARURU – CAR...
Foram encontrados 97 imagens e 97 labels.


100%|██████████| 97/97 [00:11<00:00,  8.25it/s]


Total Foreground: torch.Size([5205, 1024]), Total Background: torch.Size([19627, 1024])
Iniciando cálculo otimizado com batch size: 256
Calculando distâncias intra-classe para 5,205 vetores...


Intra-classe: 100%|██████████| 21/21 [12:12<00:00, 34.87s/it]


Calculando distâncias inter-classe para 5,205 vs 19,627 vetores...


...

100%|██████████| 2726/2726 [03:29<00:00, 12.99it/s]


Total Foreground: torch.Size([27369, 1024]), Total Background: torch.Size([670487, 1024])
Iniciando cálculo otimizado com batch size: 256
Calculando distâncias intra-classe para 27,369 vetores...


Intra-classe: 100%|██████████| 107/107 [3:47:57<00:00, 127.82s/it] 


Calculando distâncias inter-classe para 27,369 vs 670,487 vetores...


Inter-classe:  23%|██▎       | 25/107 [35:29:54<115:13:03, 5058.33s/it]

In [8]:
results = []

In [9]:
class_name, dir_class, nickname = classes[0]

print(f"Processando classe {class_name} – {nickname}...")
all_x_foreground, all_x_background = agrupar_tensores(f"{dataset_path}{dir_class}")
mean_dist_classe_classe, mean_dist_classe_not_classe, std_classe_classe, std_classe_not_classe = calcular_distancias_otimizado2(all_x_foreground, all_x_background, batch_size=4096)

print(f"Distância média {nickname}_{nickname}: {mean_dist_classe_classe}, {nickname}_not_{nickname}: {mean_dist_classe_not_classe}")
print(f"Desvio padrão {nickname}_{nickname}: {std_classe_classe}, {nickname}_not_{nickname}: {std_classe_not_classe}")

results.append({
    "class": nickname,
    #"all_foreground": all_x_foreground,
    #"all_background": all_x_background,
    "mean_dist_classe_classe": mean_dist_classe_classe,
    "mean_dist_classe_not_classe": mean_dist_classe_not_classe,
    "std_classe_classe": std_classe_classe,
    "std_classe_not_classe": std_classe_not_classe
})

del all_x_foreground, all_x_background
torch.cuda.empty_cache()

# Salvar resultados em um arquivo txt
with open("pca_soja_results-2.pkl", "wb") as f:
    pickle.dump(results, f)

Processando classe CARURU – CAR...
Foram encontrados 97 imagens e 97 labels.


100%|██████████| 97/97 [00:05<00:00, 16.60it/s]


Total Foreground: torch.Size([5205, 1024]), Total Background: torch.Size([19627, 1024])
Iniciando cálculo otimizado com batch size: 4096
Usando dispositivo: cpu
Calculando distâncias intra-classe para 5,205 vetores...


Intra-classe: 100%|██████████| 2/2 [00:00<00:00,  6.78it/s]


Calculando distâncias inter-classe para 5,205 vs 19,627 vetores...


Inter-classe: 100%|██████████| 2/2 [00:00<00:00,  2.23it/s]

Distância média CAR_CAR: 5.212090492248535, CAR_not_CAR: 5.625874042510986
Desvio padrão CAR_CAR: 0.6607735753059387, CAR_not_CAR: 0.6303556561470032


In [10]:
results

[{'class': 'CAR',
  'mean_dist_classe_classe': 5.212090492248535,
  'mean_dist_classe_not_classe': 5.625874042510986,
  'std_classe_classe': 0.6607735753059387,
  'std_classe_not_classe': 0.6303556561470032}]

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
class_name, dir_class, nickname = classes[1]

print(f"Processando classe {class_name} – {nickname}...")
all_x_foreground, all_x_background = agrupar_tensores(f"{dataset_path}{dir_class}")

mean_dist_classe_classe, mean_dist_classe_not_classe, std_classe_classe, std_classe_not_classe = calcular_distancias_otimizado2(all_x_foreground.to(device), all_x_background.to(device), batch_size=4096)

print(f"Distância média {nickname}_{nickname}: {mean_dist_classe_classe}, {nickname}_not_{nickname}: {mean_dist_classe_not_classe}")
print(f"Desvio padrão {nickname}_{nickname}: {std_classe_classe}, {nickname}_not_{nickname}: {std_classe_not_classe}")

results.append({
    "class": nickname,
    #"all_foreground": all_x_foreground,
    #"all_background": all_x_background,
    "mean_dist_classe_classe": mean_dist_classe_classe,
    "mean_dist_classe_not_classe": mean_dist_classe_not_classe,
    "std_classe_classe": std_classe_classe,
    "std_classe_not_classe": std_classe_not_classe
})

del all_x_foreground, all_x_background
torch.cuda.empty_cache()

# Salvar resultados em um arquivo txt
with open("pca_soja_results-2.pkl", "wb") as f:
    pickle.dump(results, f)

Processando classe GRAMINEA_PORTE_ALTO – GPA...
Foram encontrados 2726 imagens e 2726 labels.


  0%|          | 0/2726 [00:00<?, ?it/s]

100%|██████████| 2726/2726 [02:38<00:00, 17.20it/s]


Total Foreground: torch.Size([27369, 1024]), Total Background: torch.Size([670487, 1024])
Iniciando cálculo otimizado com batch size: 4096
Usando dispositivo: cuda:0
Calculando distâncias intra-classe para 27,369 vetores...


Intra-classe: 100%|██████████| 7/7 [00:00<00:00, 43.58it/s]


Calculando distâncias inter-classe para 27,369 vs 670,487 vetores...


Inter-classe: 100%|██████████| 7/7 [00:03<00:00,  1.97it/s]

Distância média GPA_GPA: 6.254695892333984, GPA_not_GPA: 6.467916488647461
Desvio padrão GPA_GPA: 1.230429768562317, GPA_not_GPA: 1.2287890911102295


In [14]:
class_name, dir_class, nickname = classes[2]

print(f"Processando classe {class_name} – {nickname}...")
all_x_foreground, all_x_background = agrupar_tensores(f"{dataset_path}{dir_class}")

mean_dist_classe_classe, mean_dist_classe_not_classe, std_classe_classe, std_classe_not_classe = calcular_distancias_otimizado2(all_x_foreground.to(device), all_x_background.to(device), batch_size=4096)

print(f"Distância média {nickname}_{nickname}: {mean_dist_classe_classe}, {nickname}_not_{nickname}: {mean_dist_classe_not_classe}")
print(f"Desvio padrão {nickname}_{nickname}: {std_classe_classe}, {nickname}_not_{nickname}: {std_classe_not_classe}")

results.append({
    "class": nickname,
    #"all_foreground": all_x_foreground,
    #"all_background": all_x_background,
    "mean_dist_classe_classe": mean_dist_classe_classe,
    "mean_dist_classe_not_classe": mean_dist_classe_not_classe,
    "std_classe_classe": std_classe_classe,
    "std_classe_not_classe": std_classe_not_classe
})

del all_x_foreground, all_x_background
torch.cuda.empty_cache()

# Salvar resultados em um arquivo txt
with open("pca_soja_results-2.pkl", "wb") as f:
    pickle.dump(results, f)

Processando classe GRAMINEA_PORTE_BAIXO – GPB...
Foram encontrados 1747 imagens e 1747 labels.


100%|██████████| 1747/1747 [01:40<00:00, 17.40it/s]


Total Foreground: torch.Size([15705, 1024]), Total Background: torch.Size([431527, 1024])
Iniciando cálculo otimizado com batch size: 4096
Usando dispositivo: cuda:0
Calculando distâncias intra-classe para 15,705 vetores...


Intra-classe: 100%|██████████| 4/4 [00:00<00:00, 79.42it/s]


Calculando distâncias inter-classe para 15,705 vs 431,527 vetores...


Inter-classe: 100%|██████████| 4/4 [00:01<00:00,  3.28it/s]

Distância média GPB_GPB: 5.876298904418945, GPB_not_GPB: 6.169379234313965
Desvio padrão GPB_GPB: 0.6613664627075195, GPB_not_GPB: 0.5903793573379517


In [15]:
class_name, dir_class, nickname = classes[3]

print(f"Processando classe {class_name} – {nickname}...")
all_x_foreground, all_x_background = agrupar_tensores(f"{dataset_path}{dir_class}")

mean_dist_classe_classe, mean_dist_classe_not_classe, std_classe_classe, std_classe_not_classe = calcular_distancias_otimizado2(all_x_foreground.to(device), all_x_background.to(device), batch_size=4096)

print(f"Distância média {nickname}_{nickname}: {mean_dist_classe_classe}, {nickname}_not_{nickname}: {mean_dist_classe_not_classe}")
print(f"Desvio padrão {nickname}_{nickname}: {std_classe_classe}, {nickname}_not_{nickname}: {std_classe_not_classe}")

results.append({
    "class": nickname,
    #"all_foreground": all_x_foreground,
    #"all_background": all_x_background,
    "mean_dist_classe_classe": mean_dist_classe_classe,
    "mean_dist_classe_not_classe": mean_dist_classe_not_classe,
    "std_classe_classe": std_classe_classe,
    "std_classe_not_classe": std_classe_not_classe
})

del all_x_foreground, all_x_background
torch.cuda.empty_cache()

# Salvar resultados em um arquivo txt
with open("pca_soja_results-2.pkl", "wb") as f:
    pickle.dump(results, f)

Processando classe MAMONA – MAM...
Foram encontrados 1103 imagens e 1103 labels.


100%|██████████| 1103/1103 [01:04<00:00, 16.99it/s]


Total Foreground: torch.Size([10524, 1024]), Total Background: torch.Size([271844, 1024])
Iniciando cálculo otimizado com batch size: 4096
Usando dispositivo: cuda:0
Calculando distâncias intra-classe para 10,524 vetores...


Intra-classe: 100%|██████████| 3/3 [00:00<00:00, 118.16it/s]


Calculando distâncias inter-classe para 10,524 vs 271,844 vetores...


Inter-classe: 100%|██████████| 3/3 [00:00<00:00,  8.95it/s]

Distância média MAM_MAM: 7.107694149017334, MAM_not_MAM: 7.039205551147461
Desvio padrão MAM_MAM: 4.076447486877441, MAM_not_MAM: 3.2064595222473145


In [16]:
class_name, dir_class, nickname = classes[4]

print(f"Processando classe {class_name} – {nickname}...")
all_x_foreground, all_x_background = agrupar_tensores(f"{dataset_path}{dir_class}")

mean_dist_classe_classe, mean_dist_classe_not_classe, std_classe_classe, std_classe_not_classe = calcular_distancias_otimizado2(all_x_foreground.to(device), all_x_background.to(device), batch_size=4096)

print(f"Distância média {nickname}_{nickname}: {mean_dist_classe_classe}, {nickname}_not_{nickname}: {mean_dist_classe_not_classe}")
print(f"Desvio padrão {nickname}_{nickname}: {std_classe_classe}, {nickname}_not_{nickname}: {std_classe_not_classe}")

results.append({
    "class": nickname,
    #"all_foreground": all_x_foreground,
    #"all_background": all_x_background,
    "mean_dist_classe_classe": mean_dist_classe_classe,
    "mean_dist_classe_not_classe": mean_dist_classe_not_classe,
    "std_classe_classe": std_classe_classe,
    "std_classe_not_classe": std_classe_not_classe
})

del all_x_foreground, all_x_background
torch.cuda.empty_cache()

# Salvar resultados em um arquivo txt
with open("pca_soja_results-2.pkl", "wb") as f:
    pickle.dump(results, f)

Processando classe OUTRAS_FOLHAS_LARGAS – OFL...
Foram encontrados 2999 imagens e 2999 labels.


100%|██████████| 2999/2999 [02:54<00:00, 17.16it/s]


Total Foreground: torch.Size([28817, 1024]), Total Background: torch.Size([738927, 1024])
Iniciando cálculo otimizado com batch size: 4096
Usando dispositivo: cuda:0
Calculando distâncias intra-classe para 28,817 vetores...


Intra-classe: 100%|██████████| 8/8 [00:00<00:00, 53.92it/s]


Calculando distâncias inter-classe para 28,817 vs 738,927 vetores...


Inter-classe: 100%|██████████| 8/8 [00:04<00:00,  1.90it/s]


Distância média OFL_OFL: 6.133740425109863, OFL_not_OFL: 6.291485786437988
Desvio padrão OFL_OFL: 1.0084186792373657, OFL_not_OFL: 0.8828402757644653


In [17]:
class_name, dir_class, nickname = classes[5]

print(f"Processando classe {class_name} – {nickname}...")
all_x_foreground, all_x_background = agrupar_tensores(f"{dataset_path}{dir_class}")

mean_dist_classe_classe, mean_dist_classe_not_classe, std_classe_classe, std_classe_not_classe = calcular_distancias_otimizado2(all_x_foreground.to(device), all_x_background.to(device), batch_size=4096)

print(f"Distância média {nickname}_{nickname}: {mean_dist_classe_classe}, {nickname}_not_{nickname}: {mean_dist_classe_not_classe}")
print(f"Desvio padrão {nickname}_{nickname}: {std_classe_classe}, {nickname}_not_{nickname}: {std_classe_not_classe}")

results.append({
    "class": nickname,
    #"all_foreground": all_x_foreground,
    #"all_background": all_x_background,
    "mean_dist_classe_classe": mean_dist_classe_classe,
    "mean_dist_classe_not_classe": mean_dist_classe_not_classe,
    "std_classe_classe": std_classe_classe,
    "std_classe_not_classe": std_classe_not_classe
})

del all_x_foreground, all_x_background
torch.cuda.empty_cache()

# Salvar resultados em um arquivo txt
with open("pca_soja_results-2.pkl", "wb") as f:
    pickle.dump(results, f)

Processando classe TREPADEIRA – TRE...
Foram encontrados 2320 imagens e 2320 labels.


100%|██████████| 2320/2320 [02:13<00:00, 17.44it/s]


Total Foreground: torch.Size([38800, 1024]), Total Background: torch.Size([555120, 1024])
Iniciando cálculo otimizado com batch size: 4096
Usando dispositivo: cuda:0
Calculando distâncias intra-classe para 38,800 vetores...


Intra-classe: 100%|██████████| 10/10 [00:00<00:00, 40.51it/s]


Calculando distâncias inter-classe para 38,800 vs 555,120 vetores...


Inter-classe: 100%|██████████| 10/10 [00:04<00:00,  2.34it/s]

Distância média TRE_TRE: 5.787045478820801, TRE_not_TRE: 6.364039421081543
Desvio padrão TRE_TRE: 0.7135447263717651, TRE_not_TRE: 1.0036296844482422


### Experimento 2: Foreground sera a mask das imagens da classe e Background sera a mask das imagens das outras classes